# RFM ile Müşteri Segmentasyonu
![](https://blog.hubspot.com/hubfs/what%20is%20customer%20segmentation.jpg)

## Veriyi Anlama

### İş Problemi
Bir e-ticaret şirketi müşterilerini segmentlere ayırıp bu segmentlere göre
pazarlama stratejileri belirlemek istiyor.
Örneğin şirket için çok kazançlı olan müşterileri elde tutmakiçin farklı kampanyalar, yeni müşteriler için farklı kampanyalar düzenlenmek istenmektedir.


### Veri Seti Hikayesi
* Veri seti  01/12/2009 - 09/12/2011 tarihleri arasındaki satışlarını içeriyor.
* Bu projede 2010-2011 yılları arası incelenecektir.
* Bu şirketin ürün kataloğunda hediyelik eşyalar yer almaktadır.
* Şirketin müşterilerinin büyük çoğunluğu kurumsal müşterilerdir.


### Değişkenler
* InvoiceNo: Fatura numarası. Her işleme yani faturaya ait eşsiz numara. C ile başlıyorsa iptal edilen işlem.
* StockCode: Ürün kodu. Her bir ürün için eşsiz numara.
* Description: Ürün ismi
* Quantity: Ürün adedi. Faturalardaki ürünlerden kaçar tane satıldığını ifade etmektedir.
* InvoiceDate: Fatura tarihi ve zamanı.
* UnitPrice: Ürün fiyatı (Sterlin cinsinden)
* CustomerID: Eşsiz müşteri numarası
* Country: Müşterinin yaşadığı ülke.


### Kütüphaneler

In [ ]:
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings # Uyarılar
warnings.filterwarnings("ignore")


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
online_retail = pd.read_csv("../input/online-retail-ii-uci-two-peroid/online_retail_II_2010_2011.csv", sep=";")
df = online_retail.copy()
df.head()

### Veri Önişleme

In [ ]:
def check_df(dataframe):
    print("################ Shape ####################")
    print(dataframe.shape)
    print("############### Columns ###################")
    print(dataframe.columns)
    print("############### Types #####################")
    print(dataframe.dtypes)
    print("############### Head ######################")
    print(dataframe.head())
    print("############### Tail ######################")
    print(dataframe.tail())
    print("############### Describe ###################")
    print(dataframe.describe().T)

check_df(df)

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(inplace=True)
df.isnull().sum()

In [ ]:
# Analizimizde satın almalarla ilgileniyoruz.Bu nedenle iade işlemleri veriden çıkardık.

df = df[~df["Invoice"].str.contains("C", na=False)]
df.shape

In [ ]:
# Aykırı Gözlemler

def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

replace_with_thresholds(df, "Quantity")
replace_with_thresholds(df, "Price")

### Keşifçi Veri Analizi

##### Kategorik Değişken Analizi

In [ ]:
cat_cols = [col for col in df.columns if df[col].dtypes =="O"]
cat_but_car = [col for col in df.columns if df[col].nunique() > 100 and df[col].dtypes == "O"]
cat_cols = [col for col in cat_cols if col not in cat_but_car]
cat_cols


In [ ]:
def cat_summary(dataframe, col_name, plot=False):
    print(pd.DataFrame({col_name: dataframe[col_name].value_counts(),
                        "Ratio": 100 * dataframe[col_name].value_counts() / len(dataframe)}))
    print("##########################################")
    if plot:
        fig_dims = (15, 5)
        fig, ax = plt.subplots(figsize=fig_dims)
        sns.countplot(x=dataframe[col_name], data=dataframe)
        plt.xticks(rotation = 45, ha = 'right')
        plt.show()

cat_summary(df, "Country", plot=True)

#### Nümerik Değişken Analizi

In [ ]:
num_cols = [col for col in df.columns if df[col].dtypes != 'O' and col not in "Customer ID"]
num_cols

In [ ]:
def num_summary(dataframe, numerical_col, plot=False):
    quantiles = [0.05, 0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90, 0.95, 0.99]
    print(dataframe[numerical_col].describe(quantiles).T)

    if plot:
        dataframe[numerical_col].hist(bins=20)
        plt.xlabel(numerical_col)
        plt.title(numerical_col)
        plt.show()

for col in num_cols:
    num_summary(df, col, plot=True)

In [ ]:
# eşsiz ürün sayısı
df["StockCode"].nunique()

In [ ]:
# hangi üründen kaçar adet alınmış ?
df_product = df.groupby("StockCode").agg({"Quantity":"count"})
df_product.reset_index(inplace=True)
df_product

In [ ]:
# en çok alınan 10 ürün
top_pr= df_product.sort_values(by="Quantity",ascending=False).head(10)

sns.barplot(x="StockCode", y="Quantity",data=top_pr)
plt.show()

In [ ]:
# fatura basına düsen toplam fiyatı veri setin ekleme
df["TotalPrice"] = df["Price"] * df["Quantity"]

### RFM Metriklerinin Hazırlanması

* recency: bugün ile müşterinin en son satın alma tarihi arasındaki fark, gün cinsinden
* frequency: müşterinin alışveriş sıklığı
* monetary: müşterinin ödediği toplam para


In [ ]:
# recency hesabı için analiz tarihinin belirlenmesi
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])
df["InvoiceDate"].max()
today_date = dt.datetime(2011, 12, 11)

In [ ]:
# rfm metriklerini oluşturma
rfm = df.groupby("Customer ID").agg({"InvoiceDate": lambda InvıiceDate: (today_date- InvıiceDate.max()).days,
                                    "Invoice": lambda Invoice: Invoice.nunique(),
                                    "TotalPrice": lambda TotalPrice: TotalPrice.sum()})

rfm.columns = ["recency","frequency","monetary"]
rfm.describe().T

In [ ]:
# monetary yani ödenen toplam paranın min değeri 0 işlem yapılmış para girişi olmamış
# bunları veriden çıkaröalıyım

rfm = rfm[rfm["monetary"] > 0]
rfm.describe().T

#### RFM Skorlarının Oluşturulması

In [ ]:
# recency_score
rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])
# frequency_score
rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
# monetary_score
rfm["monetary_score"] = pd.qcut(rfm["monetary"], 5, labels=[1, 2, 3, 4, 5])

# hepsinden oluşan RFM Scoru
rfm["RFM_SCORE"] = (rfm["recency_score"].astype(str) + rfm["frequency_score"].astype(str))
rfm.head(10)

#### RFM Skorlarına Göre Müşterilerin Segmentlere Ayrılması

In [ ]:
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}
rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)
rfm.head(10)


In [ ]:
# Segmentlere göre RFM ortalama ve sıklık değerlerini gruplayalım
rfm[["segment", "recency", "frequency", "monetary"]].groupby("segment").agg(["mean", "count"])


#### Segmentlerin Görselleştirilmesi

In [ ]:
sgm= rfm["segment"].value_counts()
plt.figure(figsize=(10,7))
sns.barplot(x=sgm.index,y=sgm.values)
plt.xticks(rotation=45)
plt.title('Customer Segments',color = 'blue',fontsize=15)
plt.show()

In [ ]:
labels = rfm["segment"].value_counts().index
colors = ['grey','blue','red','yellow','green','brown']
sizes = rfm["segment"].value_counts().values

# visual
plt.figure(figsize = (7,7))
plt.pie(sizes, explode=None, labels=labels, colors=colors, autopct='%1.1f%%')
plt.title('Customer Segments',color = 'blue',fontsize = 15)
plt.show()

# CLTV Prediction

### İş Problemi
Bir e-ticaret şirketi satış ve pazarlama faaliyetleri için roadmap belirlemek istemektedir.Şirketin orta uzun vadeli plan yapabilmesi için var olan müşterilerin gelecekte şirkete sağlayacakları potansiyel değerin tahmin edilmesi gerekmektedir.

### Kütüphaneler

In [ ]:
!pip install Lifetimes

In [ ]:
from sqlalchemy import create_engine
import datetime as dt
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions
from sklearn.preprocessing import MinMaxScaler

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


### Life Time Veri Yapısının Hazırlanması
* recency: müşterinin kendi özelinde son alısverisi ile ilk alısverisi arasındaki fark
* T: müşterinin şirketteki yaşı
* frequency: tekrar eden toplam satın alma sayısı
* monetary_value: satın alma başına ortalama kazanç


In [ ]:
cltv_df = df.groupby('Customer ID').agg({'InvoiceDate': [lambda date: (date.max() - date.min()).days,
                                                         lambda date: (today_date - date.min()).days],
                                         'Invoice': lambda num: num.nunique(),
                                         'TotalPrice': lambda TotalPrice: TotalPrice.sum()})


cltv_df.columns = cltv_df.columns.droplevel(0)
cltv_df.columns = ['recency', 'T', 'frequency', 'monetary']
cltv_df.head()

In [ ]:
# monetary değerini toplam totalPrice olarak hesaplamıştık.
# bu aşamada moneary değerini satın alma başına ortalama kazanç olarak ifade edeceğiz
cltv_df["monetary"] = cltv_df["monetary"] / cltv_df["frequency"]

# monetary nin sıfırdan büyük olanlarının seçelimesi
cltv_df = cltv_df[cltv_df["monetary"] > 0]

# BGNBD için recency ve T'nin haftalık cinsten ifade edilmesi
cltv_df["recency"] = cltv_df["recency"] / 7
cltv_df["T"] = cltv_df["T"] / 7

# frequency nin 1 den büyük olanlarının seçilmesi
cltv_df = cltv_df[(cltv_df['frequency'] > 1)]
cltv_df.head()

### BG-NBD Modelinin Kurulması
* Bu modelle amacım satın alma sayısını olasılıksal olarak tahmin etmek

In [ ]:
bgf = BetaGeoFitter(penalizer_coef=0.001)
bgf.fit(cltv_df['frequency'],
        cltv_df['recency'],
        cltv_df['T'])

In [ ]:
# 1 haftalık beklenen satın alma (transection)
cltv_df["expected_purc_1_week"] = bgf.predict(1,
                                               cltv_df['frequency'],
                                               cltv_df['recency'],
                                               cltv_df['T'])

cltv_df.sort_values("expected_purc_1_week", ascending=False).head(10)

In [ ]:
# 1 aylık beklenen satın alma
cltv_df["expected_purc_1_month"] = bgf.predict(4,
                                               cltv_df['frequency'],
                                               cltv_df['recency'],
                                               cltv_df['T'])

cltv_df.sort_values("expected_purc_1_month", ascending=False).head(10)

### GAMMA-GAMMA Modelinin Kurulması
* Bu modelle amaç average ptofiti yani işlem başına ortalama karı olasılıksal olarak modellemek

In [ ]:
ggf = GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(cltv_df['frequency'], cltv_df['monetary'])

In [ ]:
cltv_df["expected_average_profit"] = ggf.conditional_expected_average_profit(cltv_df['frequency'],
                                                                             cltv_df['monetary'])

cltv_df.sort_values("expected_average_profit", ascending=False).head(20)

* İncelendiğinde kar getirisi en yüksek olan kişiden daha fazla alısveris yapan biri daha az kar getirmiş.Burada terslik var demek ki sadece buraya göre bir cltv pred. yapamam bu iki modeli çarpıstırmam gerek.


### BG-NBD ve GG modeli ile CLTV'nin çarpıştırılması


In [ ]:
cltv = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency'],
                                   cltv_df['T'],
                                   cltv_df['monetary'],
                                   time=6,  # 6 aylık
                                   freq="W",  # T'nin frekans bilgisi.
                                   discount_rate=0.01)

In [ ]:
# ID indexte bunu indexten çıkaralım
cltv = cltv.reset_index()
# ana tablom ile tahmin değerleri tablosunu birleştiriyorum
cltv_final = cltv_df.merge(cltv, on="Customer ID", how="left")
# azalan şekilde sıralayalım
cltv_final.sort_values(by="clv", ascending=False).head(10)

In [ ]:
# 1 aylık CLTV hesaplayalım:
cltv_1 = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency'],
                                   cltv_df['T'],
                                   cltv_df['monetary'],
                                   time=1,  # 1 aylık
                                   freq="W",  # T'nin frekans bilgisi
                                   discount_rate=0.01)

cltv_1.head()
cltv_1= cltv_1.reset_index()
cltv_1 = cltv_df.merge(cltv_1, on="Customer ID", how="left")
cltv_1.sort_values(by="clv", ascending=False).head(10)


In [ ]:
# 12 aylık CLTV hesaplayalım:

cltv_12 = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency'],
                                   cltv_df['T'],
                                   cltv_df['monetary'],
                                   time=12,  # 1 aylık
                                   freq="W",  # T'nin frekans bilgisi
                                   discount_rate=0.01)

cltv_12.head()
cltv_12 = cltv_12.reset_index()
cltv_12 = cltv_df.merge(cltv_12, on="Customer ID", how="left")
cltv_12.sort_values(by="clv", ascending=False).head(10)


* 1 aylık ve 12 aylık tahminlere bakıldığında aynı müşterilerin iki durumda da clv tahminlerinin yüksek olduğunu görüyorum.
* Sadece bazı müşteriler yer değiştirmiş,ama çok fazla fark yok.
* Yani müşteriler 12 ay içinde alısveris alıskalnlıklarını aynı şekilde devam ettirmiş.


#### CLTV Tahminlerine Göre Müşterilerin Segmentlere Ayrılması

In [ ]:
# clv tahminlerinin 0-1 aralığına normalize edilmesi
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(cltv_final[["clv"]])
cltv_final["scaled_clv"] = scaler.transform(cltv_final[["clv"]])

cltv_final.sort_values(by="scaled_clv", ascending=False).head()

In [ ]:
# Müşterilerin Segmentlere Ayrılması
cltv_final["segment"] = pd.qcut(cltv_final["scaled_clv"], 4, labels=["D", "C", "B", "A"])
cltv_final.head()

cltv_final.head()


In [ ]:
# segnmentlerin incelenmesi
cltv_final.groupby("segment").agg({"count", "mean", "sum"})